In [95]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2
from IPython.display import Image as CImage

In [106]:
train_path='/Users/cj/Downloads/t_data/training'
pred_path='/Users/cj/Downloads/t_data/test'

In [97]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [98]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [107]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=33)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=33, bias=True)
)

In [108]:

#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [140]:
#prediction function
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
#     print(index)
    pred=classes[index]
    
    return pred

In [141]:
classes

['1', '10', '100', '1000', '10000', '20', '20000', '5', '50', '500', '5000']

In [144]:
images_path=glob.glob(pred_path+'/*.jpg')

In [145]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)
    print(i[i.rfind('/')+1:])
#     print(prediction(i,transformer))

img_10.jpg
img_1.jpg
IMG_3455.jpg
IMG_3454.jpg
img_11.jpg
img_13.jpg
img_12.jpg
img_16.jpg
img_7.jpg
8k262.jpg
IMG_3453.jpg
IMG_3452.jpg
img_6.jpg
img_17.jpg
img_15.jpg
img_4.jpg
IMG_3451.jpg
img_5.jpg
img_14.jpg
img_19.jpg
img_8.jpg
img_9.jpg
img_18.jpg
ICANN69 Custom Zoom virtual background_02_icann-meeting.jpg


In [146]:
pred_dict

{'img_10.jpg': '500',
 'img_1.jpg': '500',
 'IMG_3455.jpg': '5000',
 'IMG_3454.jpg': '500',
 'img_11.jpg': '50',
 'img_13.jpg': '1000',
 'img_12.jpg': '1000',
 'img_16.jpg': '20000',
 'img_7.jpg': '1000',
 '8k262.jpg': '10',
 'IMG_3453.jpg': '5000',
 'IMG_3452.jpg': '20000',
 'img_6.jpg': '10000',
 'img_17.jpg': '20000',
 'img_15.jpg': '20000',
 'img_4.jpg': '5000',
 'IMG_3451.jpg': '5000',
 'img_5.jpg': '500',
 'img_14.jpg': '5000',
 'img_19.jpg': '20000',
 'img_8.jpg': '10000',
 'img_9.jpg': '10000',
 'img_18.jpg': '500',
 'ICANN69 Custom Zoom virtual background_02_icann-meeting.jpg': '10'}

In [129]:
torch.save(model.state_dict(), 'weights_only.pth')